# Script for merging Camera and Lidar Data

In [15]:
import numpy as np
import open3d as o3d
import cv2
import os
import glob

# Directories
camera_dir = "/Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw"
lidar_dir = "/Users/amosweckstrom/Desktop/ÅBOAT/LidarData"
output_dir = os.path.join(camera_dir, "lidar_overlay")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Camera intrinsic parameters
image_width, image_height = 720, 480
fov_angle = 90
focal_length = image_width / (2 * np.tan(np.radians(fov_angle) / 2))
cx, cy = image_width / 2, image_height / 2
K = np.array([[focal_length, 0, cx],
              [0, focal_length, cy],
              [0, 0, 1]])

# Camera translation in world frame (from the provided config)
cam_tx, cam_ty, cam_tz = 300.001, 0.0, 260.0

# Rotation from world frame to camera frame:
# World -> Camera:
# X_world -> Z_cam
# Y_world -> X_cam
# Z_world -> -Y_cam
R = np.array([
    [0,  1,  0],
    [0,  0, -1],
    [1,  0,  0]
])

# Gather image and Lidar files
image_files = glob.glob(os.path.join(camera_dir, "processed_image_*.png"))
image_files.sort()

lidar_files = glob.glob(os.path.join(lidar_dir, "pcl*.pcd"))
lidar_files.sort()

# Parse the numeric index from filenames.
# Images are like processed_image_0000.png -> index 0
# Lidar are like pcl0.pcd -> index 0

def extract_image_index(filename):
    # Extract from processed_image_XXXX.png
    # Example: processed_image_0000.png -> index 0
    base = os.path.basename(filename)
    name_no_ext, _ = os.path.splitext(base)
    # name_no_ext = "processed_image_0000"
    parts = name_no_ext.split("_")
    if len(parts) < 3:
        return None
    idx_str = parts[-1]
    if idx_str.isdigit():
        return int(idx_str)
    return None

def extract_lidar_index(filename):
    # Extract from pclX.pcd
    # Example: pcl0.pcd -> index 0
    base = os.path.basename(filename)
    name_no_ext, _ = os.path.splitext(base)
    # name_no_ext = "pcl0"
    if name_no_ext.startswith("pcl"):
        idx_str = name_no_ext[3:]
        if idx_str.isdigit():
            return int(idx_str)
    return None

# Create a map from index to lidar file
lidar_index_map = {}
for lf in lidar_files:
    li = extract_lidar_index(lf)
    if li is not None:
        lidar_index_map[li] = lf

# Process each image, find corresponding Lidar
for img_path in image_files:
    frame_index = extract_image_index(img_path)
    if frame_index is None:
        print(f"Skipping {img_path}, cannot parse frame index.")
        continue

    if frame_index not in lidar_index_map:
        print(f"No matching Lidar file for image {img_path} (index: {frame_index}). Skipping.")
        continue

    lidar_file = lidar_index_map[frame_index]

    # Load camera image
    camera_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if camera_img is None:
        print(f"Cannot load image at {img_path}, skipping.")
        continue

    camera_img = cv2.resize(camera_img, (image_width, image_height), interpolation=cv2.INTER_LINEAR)

    # Load LiDAR point cloud
    pcd = o3d.io.read_point_cloud(lidar_file)
    points = np.asarray(pcd.points)

    # Transform points into camera frame
    points_translated = points - np.array([cam_tx, cam_ty, cam_tz])
    points_cam = points_translated @ R.T

    # Project and overlay LiDAR points
    circle_radius = 5    # Increase point size
    circle_thickness = -1
    point_color = (0, 0, 255) # Red

    for Xc, Yc, Zc in points_cam:
        # Only consider points in front of camera
        if Zc <= 0:
            continue
        uv = K @ np.array([Xc, Yc, Zc])
        u, v = uv[0] / uv[2], uv[1] / uv[2]

        u_int, v_int = int(round(u)), int(round(v))
        if 0 <= u_int < image_width and 0 <= v_int < image_height:
            cv2.circle(camera_img, (u_int, v_int), radius=circle_radius, color=point_color, thickness=circle_thickness)

    # Save the overlayed image
    output_path = os.path.join(output_dir, f"overlay_{frame_index}.png")
    cv2.imwrite(output_path, camera_img)
    print(f"Saved overlayed image: {output_path}")


Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_0.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_1.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_2.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_3.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_4.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_5.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_6.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/CameraData/captured_images_raw/lidar_overlay/overlay_7.png
Saved overlayed image: /Users/amosweckstrom/Desktop/ÅBOAT/Camera